# Use pretrained model for test predictions

In [1]:
!pip3 install kenlm
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184422 sha256=1c4af3d54acb7ba0ef734e3f426e7fa9dfa637f8526c95b9450fe24bc9f2c6e9
  Stored in directory: /root/.cache/pip/wheels/fd/80/e0/18f4148e863fb137bd87e21ee2bf423b81b3ed6989dab95135
Successfully built kenlm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.4 MB/s eta 0:00:00
     ━

In [2]:
import torch

from utils import SR
from datasets import Audio, load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# DL data
dataset_name = "lucas-meyer/asr_af"
af_test_set = load_dataset(dataset_name, split="test")
af_test_set = af_test_set.cast_column("audio", Audio(sampling_rate=SR))

# DL model
repo_name = "lucas-meyer/wav2vec2-xls-r-300m-with-LM-asr_af"
model = Wav2Vec2ForCTC.from_pretrained(repo_name)
processor = Wav2Vec2Processor.from_pretrained(repo_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3325 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/511 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/585 [00:00<?, ? examples/s]

In [3]:
for i in range(100):
    input_values = processor(af_test_set[i]["audio"]["array"], sampling_rate=SR).input_values[0]
    input_dict = processor(input_values,
                           sampling_rate=SR,
                           return_tensors="pt",
                           padding=True)

    logits = model(input_dict.input_values).logits
    logits = logits.detach()
    pred_ids = torch.argmax(logits, dim=-1)[0]

    pred = processor.decode(pred_ids)
    true = af_test_set[i]["transcription"].lower()

    print(f"Test {i}:")
    print(f"  - pred: {pred}")
    print(f"  - true: {true}\n")

Test 0:
  - pred: waarom is dit positief negatief of neeutraaal
  - true: waarom is dit positief negatief of neutraal

Test 1:
  - pred: dus is die woord measurer baie naby aan leisure
  - true: dus is die woord measure baie naby aan leisure

Test 2:
  - pred: die musiekgroep zefmaar zen het meer as 25 miljoen albums wêreldwyd verkoop
  - true: die muskiekgroep zef maar zen het meer as 25 miljoen albums wêreldwyd verkoop

Test 3:
  - pred: thus hat te row worldenes proepiedhim for this dayy
  - true: thus had the raw wilderness prepared him for this day

Test 4:
  - pred: uiteindelik was daar geen meersprake van dosnies en emirate nie
  - true: uiteindelik was daar geen meer sprake van dominees en emirate nie

Test 5:
  - pred: vyf en vyftig twee een en twintig
  - true: vyf en vyftig twee een en twintig

Test 6:
  - pred: dit is minus 1 grade celsius in bocsburg
  - true: dit is minus 1 grade celsius in boksburg

Test 7:
  - pred: postma het om tot almal se verbasing losgemaak van ker